# data download

In [1]:
import argparse
import json
from tqdm import tqdm


def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}



parser = argparse.ArgumentParser()
parser.add_argument("--data_path", type=str, default="C:/Users/1/Documents/Data/alpaca_data.json")
parser.add_argument("--save_path", type=str, default="C:/Users/1/Documents/Data/alpaca_data.jsonl")

data_path = 'C:/Users/1/Documents/Data/alpaca_data.json'
save_path = 

args = parser.parse_args()
with open(args.data_path) as f:
    examples = json.load(f)

with open(args.save_path, 'w') as f:
    for example in tqdm(examples, desc="formatting.."):
        f.write(json.dumps(format_example(example)) + '\n')

usage: ipykernel_launcher.py [-h] [--data_path DATA_PATH]
                             [--save_path SAVE_PATH]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"fa2dfc82-ea38-4558-a744-541b13691d00" --shell=9007 --transport="tcp" --iopub=9009 --f=c:\Users\1\AppData\Roaming\jupyter\runtime\kernel-v2-2020RbtnE7fvIYBC.json


SystemExit: 2

c:\Users\1\miniconda3\envs\LLM_py310\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import argparse
import json
from tqdm import tqdm

import datasets
import transformers


def preprocess(tokenizer, config, example, max_seq_length):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True)
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        add_special_tokens=False)
    input_ids = prompt_ids + target_ids + [config.eos_token_id]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}


def read_jsonl(path, max_seq_length, skip_overlength=False):
    model_name = "THUDM/chatglm-6b"
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_name, trust_remote_code=True)
    config = transformers.AutoConfig.from_pretrained(
        model_name, trust_remote_code=True, device_map='auto')
    with open(path, "r") as f:
        for line in tqdm(f.readlines()):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature



parser = argparse.ArgumentParser()
parser.add_argument("--jsonl_path", type=str, default="data/alpaca_data.jsonl")
parser.add_argument("--save_path", type=str, default="data/alpaca")
parser.add_argument("--max_seq_length", type=int, default=384)
parser.add_argument("--skip_overlength", type=bool, default=False)
args = parser.parse_args()

dataset = datasets.Dataset.from_generator(
    lambda: read_jsonl(args.jsonl_path, args.max_seq_length, args.skip_overlength)
)
dataset.save_to_disk(args.save_path)